In [1]:
import requests
from urllib import parse
from urllib.parse import quote, urlencode
import datetime
from datetime import datetime, timezone, date

from oauthlib.oauth2 import WebApplicationClient

import json
import tiktoken
import os

import xml.etree.ElementTree as ET

import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timezone, date


import bs4 #BeautifulSoup 
from bs4 import BeautifulSoup as bs
import re

# Get Inoreader news using OAuth authentication

### Get authorization Uri

In [9]:


AUTH_URL      = 'https://www.inoreader.com/oauth2/auth'
CLIENT_ID     = '1000003205'
CLIENT_SECRET = 'QUgOhKZQFoEPwubkk67fnQNKbn1tpr4O'
GRANT_TYPE    = 'authorization_code'
ITEMS_PER_PAGE = 100  # 100 is the maximum value according to the documentation
RANDOM_STRING = 'ankscajkjnsckljU8127481924asdasdsdfqww'  # Some random string
REDIRECT_URI  = 'http://localhost:5000/oauth2callback'  # This one doesn't really matter
SCOPE         = 'read'



TOKEN_URL     = 'https://www.inoreader.com/oauth2/token'


# date from collect 
date_time = datetime(2025, 9, 18, 0, 0, tzinfo=timezone.utc)
# prepare timestamps
timestamp_seconds = int(date_time.timestamp())

client = WebApplicationClient(CLIENT_ID)

url = client.prepare_request_uri(
  AUTH_URL,
  redirect_uri = REDIRECT_URI,
  scope = SCOPE,
  state = RANDOM_STRING,
)

print('Navigate to url printed below:')

url

Navigate to url printed below:


'https://www.inoreader.com/oauth2/auth?response_type=code&client_id=1000003205&redirect_uri=http%3A%2F%2Flocalhost%3A5000%2Foauth2callback&scope=read&state=ankscajkjnsckljU8127481924asdasdsdfqww'

### Get access token

* Navigate to Url
* Authorize Application
* After the redirect occurs, copy `code` value from Url string
* Update the `CODE` variable in the cell below 

In [10]:
# Copy code from redirected url
CODE = 'dd5529e31b4beab63977bab040bb510121cc584d'

data = dict(
  code = CODE,
  redirect_uri = REDIRECT_URI,
  client_id = CLIENT_ID,
  client_secret = CLIENT_SECRET,
  grant_type = GRANT_TYPE,
)

token = requests.post(TOKEN_URL, data=data)

access_token = token.json()['access_token']

print(f'Access token: {access_token}')

Access token: 3b76554a8defae78a9dba611aeb44003696be5e8


In [51]:
token

<Response [400]>

In [53]:
import os
import logging
from typing import Dict, Optional
import requests

# Configure logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

def exchange_code_for_token(
    code: str,
    token_url: str,
    client_id: str,
    client_secret: str,
    redirect_uri: str,
    grant_type: str = "authorization_code",
    timeout: float = 10.0,
    use_basic_auth: bool = True,
    session: Optional[requests.Session] = None,
) -> Dict[str, object]:
    """
    Exchanges an authorization code for an access token.

    Args:
        code (str): The authorization code.
        token_url (str): The token endpoint URL.
        client_id (str): The client ID.
        client_secret (str): The client secret.
        redirect_uri (str): The redirect URI.
        grant_type (str, optional): The grant type. Defaults to "authorization_code".
        timeout (float, optional): Request timeout in seconds. Defaults to 10.0.
        use_basic_auth (bool, optional): Whether to use HTTP Basic Auth. Defaults to True.
        session (Optional[requests.Session], optional): A requests session. Defaults to None.

    Returns:
        Dict[str, object]: The token response data.

    Raises:
        RuntimeError: If the token request fails or the response is invalid.
    """
    payload = {
        "grant_type": grant_type,
        "code": code,
        "redirect_uri": redirect_uri,
    }
    headers = {"Accept": "application/json"}
    auth = (client_id, client_secret) if use_basic_auth else None

    if not use_basic_auth:
        # Fallback: some providers require client credentials in the body
        payload["client_id"] = client_id
        payload["client_secret"] = client_secret

    sess = session or requests
    try:
        resp = sess.post(token_url, data=payload, headers=headers, auth=auth, timeout=timeout)
        resp.raise_for_status()
        token_data = resp.json()
    except requests.exceptions.RequestException as e:
        raise RuntimeError(f"Token request failed: {e}") from e
    except ValueError as e:
        # Response was not valid JSON
        raise RuntimeError(f"Token endpoint did not return valid JSON. Body: {resp.text[:300]}") from e

    if "access_token" not in token_data:
        raise RuntimeError(f"Response missing access_token: {token_data}")

    return token_data

def mask_token(token: str, show: int = 4) -> str:
    """
    Masks a token for logging purposes.

    Args:
        token (str): The token to mask.
        show (int, optional): Number of characters to show at the end. Defaults to 4.

    Returns:
        str: The masked token.
    """
    return ("*" * max(len(token) - show, 0)) + token[-show:]

if __name__ == "__main__":
    CODE = os.environ.get("OAUTH_CODE") or "replace-with-code"
    TOKEN_URL = os.environ.get("AUTH_URL") or "https://www.inoreader.com/oauth2/auth"
    CLIENT_ID = os.environ.get("CLIENT_ID") or "1000003205"
    CLIENT_SECRET = os.environ.get("CLIENT_SECRET") or "QUgOhKZQFoEPwubkk67fnQNKbn1tpr4O"
    REDIRECT_URI = os.environ.get("REDIRECT_URI") or "http://localhost:5000/oauth2callback"

    

    token = exchange_code_for_token(
        code=CODE,
        token_url=TOKEN_URL,
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET,
        redirect_uri=REDIRECT_URI,
        use_basic_auth=True,  # Prefer HTTP Basic per RFC 6749 §2.3.1 if supported by provider
    )

    access_token = str(token["access_token"])
    logger.info("Access token: %s", mask_token(access_token))

RuntimeError: Token request failed: Expecting value: line 1 column 1 (char 0)

### Load items

In [11]:
#STREAM_ID     = 'user/-/label/AI_News'  # '-' is a current user
STREAM_ID     = 'user/-/tag/AI_News'  # '-' is a current user

auth_header = f'Bearer {access_token}'

encoded_stream_id = parse.parse_qs(STREAM_ID)
 
# Base URL for the API request
base_url = f'https://www.inoreader.com/reader/api/0/stream/contents/{encoded_stream_id}'

headers = {'Authorization': auth_header}

default_params = dict(
    n = ITEMS_PER_PAGE,
    ot = timestamp_seconds #### DD ######
)

items = []

def fetch_page(url, params, headers):
    """Fetch a page from Inoreader API and return (items, continuation)."""
    try:
        resp = requests.get(url, params=params, headers=headers)
    except Exception as e:
        print(f'Can not proceed because of: {e}')
        return None, None

    if resp.status_code != requests.status_codes.codes.OK:
        print(f'Failed to load response because of: `{resp.status_code}: {resp.content}`')
        return None, None

    data = resp.json()
    return data.get('items', []), data.get('continuation')

# First page
items_page, continuation = fetch_page(base_url, default_params, headers)
if items_page:
    items.extend(items_page)

# Fetch subsequent pages while continuation token is present
while continuation:
  params = dict(**default_params, c=continuation)
  items_page, continuation = fetch_page(base_url, params, headers)
  if items_page:
    items.extend(items_page)
  else:
    break

print(f'Loaded: {len(items)}')

Loaded: 8571


# Get text from json

In [18]:
items[99]

{'crawlTimeMsec': '1761642951495',
 'timestampUsec': '1761642951495427',
 'id': 'tag:google.com,2005:reader/item/0000000af1233908',
 'categories': ['user/1005282412/state/com.google/reading-list'],
 'title': 'A dynamic Fermatean fuzzy approach for multistage decision-making in investment benefit assessment of smart grids',
 'published': 1761609600,
 'updated': 0,
 'canonical': [{'href': 'https://link.springer.com/article/10.1007/s10462-025-11386-5'}],
 'alternate': [{'href': 'https://link.springer.com/article/10.1007/s10462-025-11386-5',
   'type': 'text/html'}],
 'summary': {'direction': 'ltr',
  'content': '<p><img src="https://static-content.springer.com/image/art%3A10.1007%2Fs10462-025-11386-5/MediaObjects/10462_2025_11386_Fig1_HTML.png" alt="10462_2025_11386_Fig1_HTML.png"></p>'},
 'author': '',
 'likingUsers': [],
 'comments': [],
 'commentsNum': -1,
 'annotations': [],
 'origin': {'streamId': 'feed/keyword-monitoring-1007989224',
  'title': 'Latest Results for Artificial Intelli

In [13]:
all_items = items
articles = []

for item in all_items:
    canonical_list = item.get('canonical', [])
    if isinstance(canonical_list, list) and len(canonical_list) > 0:
        canonical_url = canonical_list[0].get('href', 'NA')
    else:
        # Fallback to alternate if canonical missing
        alt_list = item.get('alternate', [])
        canonical_url = alt_list[0].get('href', 'NA') if isinstance(alt_list, list) and len(alt_list) > 0 else 'NA'

    content_html = item.get('summary', {}).get('content', 'NA')

    articles.append({
        'title': item.get('title', 'NA'),
        'published': item.get('published', 'NA'),
        'canonical': canonical_url,
        'content': content_html,
        'sourse': item.get('origin', {}).get('title', 'NA'),  # using 'sourse' as requested
        'tags': item.get('tags', 'NA')  # keep if you still want tags
    })

df = pd.DataFrame(articles)

# BeautifulSoup cleanup
df['content'] = [bs(text).get_text() if isinstance(text, str) else 'NA' for text in df['content']]
df['content'] = [re.sub(r'\n\n+', '\n', ct) if isinstance(ct, str) else 'NA' for ct in df['content']]
df['content'] = [re.sub(r' {2,}', ' ', ct) if isinstance(ct, str) else 'NA' for ct in df['content']]
df['content'] = df['content'].replace(r'\s+', ' ', regex=True)

# Add token count column
def num_tokens_from_string(string, encoding_name):
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string if isinstance(string, str) else '', disallowed_special=()))
    return num_tokens

df['token_count'] = df['content'].apply(lambda text: num_tokens_from_string(text, "cl100k_base"))
print("Number texts before duplicates cleaning: ", len(df))

df1 = df.sort_values(by=['published'])
print('From: ', df1['published'].min())
print('To: ', df1['published'].max())

# Drop duplicated posts (use canonical instead of url)
df2 = df1.drop_duplicates(subset=['title', 'canonical'])
df2 = df2.sort_values(by=['published'], ascending=False)

print("Number texts after duplicates cleaning: ", len(df2))

df2

Number texts before duplicates cleaning:  8571
From:  0
To:  1761664205
Number texts after duplicates cleaning:  5336


,title,published,canonical,content,sourse,tags,token_count
2,OpenAI completes conversion to for-profit busi...,1761664205,https://www.theguardian.com/technology/2025/oc...,Restructuring paves way for ChatGPT maker to m...,Artificial intelligence (AI) | The Guardian,NA,104
4,Finding return on AI investments across indust...,1761663633,https://www.technologyreview.com/2025/10/28/11...,The market is officially three years post Chat...,Artificial intelligence – MIT Technology Revie...,NA,1241
8,Truth Systems Bags $4m To Enforce AI Guardrails,1761663600,https://www.artificiallawyer.com/2025/10/28/tr...,"Truth Systems, a California start-up advised b...",Artificial Lawyer,NA,33
10,‘A stomach of steel’: amateur investors ride o...,1761662544,https://www.theguardian.com/business/2025/oct/...,As one 23-year-old buys a house with his winni...,Artificial intelligence (AI) | The Guardian (A...,NA,86
21,"Water Cooler Small Talk, Ep. 9: What “Thinking...",1761661800,https://towardsdatascience.com/water-cooler-sm...,Understanding how AI models “reason” and why i...,Towards Data Science - Medium (News AI filter),NA,54
...,...,...,...,...,...,...,...
3614,What we’re optimizing ChatGPT for,1754265600,https://openai.com/index/optimizing-chatgpt,We build ChatGPT to help you thrive in all the...,OpenAI Blog (AI_Filtered),NA,48
7074,,0,https://www.inoreader.com/article/3a9c6e77631b...,,Towards Data Science - Medium,NA,0
457,,0,https://www.inoreader.com/article/3a9c6e775430...,,Towards AI - Medium,NA,0
455,,0,https://www.inoreader.com/article/3a9c6e775430...,,Towards AI - Medium (News AI filter),NA,0


In [45]:
df3 = df2[df2['content']!=''].copy()
print("Rows with all tokens:", len(df3))

df3 = df3.loc[df3['token_count'] > 149]
print("Rows with tokens>149:", len(df3))

df3['published'] = pd.to_datetime(df3['published'], unit='s')
print('From: ', min(df3['published']))
print('To: ', max(df3['published']))
df3

Rows with all tokens: 5177
Rows with tokens>149: 1459
From:  2025-09-15 14:49:01
To:  2025-10-28 15:00:33


,title,published,canonical,content,sourse,tags,token_count
4,Finding return on AI investments across indust...,2025-10-28 15:00:33,https://www.technologyreview.com/2025/10/28/11...,The market is officially three years post Chat...,Artificial intelligence – MIT Technology Revie...,NA,1241
23,OpenAI completed its for-profit restructuring ...,2025-10-28 13:38:55,https://www.theverge.com/news/807875/openai-mi...,Microsoft Chairman and Chief Executive Office...,The Verge - All Posts (News AI filter),NA,557
32,Tech chiefs tell Trump to call off troops – wi...,2025-10-28 13:22:19,https://www.theguardian.com/technology/2025/oc...,In this week’s newsletter: Silicon Valley bill...,Artificial intelligence (AI) | The Guardian,NA,253
35,Adobe Max 2025: all the latest creative tools ...,2025-10-28 13:16:17,https://www.theverge.com/news/807867/adobe-max...,Adobe has kicked off its annual Max design co...,The Verge - All Posts,NA,245
43,Fast vs. Slow: How (and When) to Make Models T...,2025-10-28 12:26:53,https://pub.towardsai.net/fast-vs-slow-how-and...,Everyone in the Room Went “Hmm”! Except me…Our...,Towards AI - Medium (News AI filter),NA,1921
...,...,...,...,...,...,...,...
8565,The new Oakley Meta glasses are what athletes ...,2025-09-18 00:14:36,https://www.theverge.com/tech/779452/oakley-me...,This was what I had been expecting from Oakle...,The Verge - All Posts,NA,176
8568,Meta&rsquo;s new Ray-Ban smart glasses have tw...,2025-09-18 00:07:37,https://www.theverge.com/news/780012/meta-ray-...,Meta has announced a new generation of its Ra...,The Verge - All Posts (AI_Filtered),NA,402
6158,Beyond “Looks Good to Me”: How to Quantify LLM...,2025-09-17 21:31:29,https://pub.towardsai.net/beyond-looks-good-to...,The Production HurdleThe greatest challenge fa...,Towards AI - Medium (News AI filter),NA,1315
7465,AI-Driven “Vibe Coding”: A New Era of Developm...,2025-09-15 15:16:21,https://pub.towardsai.net/ai-driven-vibe-codin...,Is AI developer a “Vibe Coder”?The term “vibe ...,Towards AI - Medium (News AI filter),NA,1776


In [48]:
inoreader2 = df3.rename(columns={'published': 'date_published', 'canonical': 'url', 'content':'content_html'})
inoreader2 = inoreader2[['title', 'url', 'content_html', 'date_published', 'token_count']]
inoreader2

,title,url,content_html,date_published,token_count
4,Finding return on AI investments across indust...,https://www.technologyreview.com/2025/10/28/11...,The market is officially three years post Chat...,2025-10-28 15:00:33,1241
23,OpenAI completed its for-profit restructuring ...,https://www.theverge.com/news/807875/openai-mi...,Microsoft Chairman and Chief Executive Office...,2025-10-28 13:38:55,557
32,Tech chiefs tell Trump to call off troops – wi...,https://www.theguardian.com/technology/2025/oc...,In this week’s newsletter: Silicon Valley bill...,2025-10-28 13:22:19,253
35,Adobe Max 2025: all the latest creative tools ...,https://www.theverge.com/news/807867/adobe-max...,Adobe has kicked off its annual Max design co...,2025-10-28 13:16:17,245
43,Fast vs. Slow: How (and When) to Make Models T...,https://pub.towardsai.net/fast-vs-slow-how-and...,Everyone in the Room Went “Hmm”! Except me…Our...,2025-10-28 12:26:53,1921
...,...,...,...,...,...
8565,The new Oakley Meta glasses are what athletes ...,https://www.theverge.com/tech/779452/oakley-me...,This was what I had been expecting from Oakle...,2025-09-18 00:14:36,176
8568,Meta&rsquo;s new Ray-Ban smart glasses have tw...,https://www.theverge.com/news/780012/meta-ray-...,Meta has announced a new generation of its Ra...,2025-09-18 00:07:37,402
6158,Beyond “Looks Good to Me”: How to Quantify LLM...,https://pub.towardsai.net/beyond-looks-good-to...,The Production HurdleThe greatest challenge fa...,2025-09-17 21:31:29,1315
7465,AI-Driven “Vibe Coding”: A New Era of Developm...,https://pub.towardsai.net/ai-driven-vibe-codin...,Is AI developer a “Vibe Coder”?The term “vibe ...,2025-09-15 15:16:21,1776


# Concat with previously collected news

In [50]:
# Get data 
file_path = r'C:\Users\Denis_Davydov2\OneDrive - EPAM\Prophet_AI_docs\Datasets\AI_skills'
file_name = r'\Inoreader_API_Aug_Sept_2025.csv'


inoreader1 = pd.read_csv(file_path+file_name)
print("File read: ", file_name)

print('From: ', min(inoreader1['date_published']))
print('To: ', max(inoreader1['date_published']))
inoreader1 = inoreader1[['title', 'url', 'content_html', 'date_published', 'token_count']]

print("Rows total: ", len(inoreader1))
inoreader1 = inoreader1.loc[inoreader1['token_count'] > 149]
print("Rows with tokens>149: ", len(inoreader1))

File read:  \Inoreader_API_Aug_Sept_2025.csv
From:  2025-08-02T02:01:31+00:00
To:  2025-09-18T16:34:09+00:00
Rows total:  2848
Rows with tokens>149:  1547


In [59]:
innoreader_all = pd.concat([inoreader1, inoreader2])
innoreader_all


,title,url,content_html,date_published,token_count
1,Designing Multimodal AI Search Engines for Sma...,https://pub.towardsai.net/designing-multimodal...,"How AI redefines product discovery, search, an...",2025-08-13T20:49:41+00:00,12340
3,Is Amazon testing a cheaper color Kindle?,https://www.theverge.com/news/758951/amazon-ki...,A Redditor has shared a few images of what th...,2025-08-13T20:33:25+00:00,294
4,Microplastics are everywhere — including in th...,https://www.theverge.com/report/758868/plastic...,"A close-up of artwork by Canadian artist, act...",2025-08-13T20:13:25+00:00,186
7,PwC and AWS Build Responsible AI with Automate...,https://aws.amazon.com/blogs/machine-learning/...,This is a guest post co-written with Scott Lik...,2025-08-13T19:44:03+00:00,2377
9,Apple’s plan for AI could make Siri the animat...,https://www.theverge.com/news/758931/apple-ai-...,Apple is developing a bunch of products and f...,2025-08-13T19:41:12+00:00,471
...,...,...,...,...,...
8565,The new Oakley Meta glasses are what athletes ...,https://www.theverge.com/tech/779452/oakley-me...,This was what I had been expecting from Oakle...,2025-09-18 00:14:36,176
8568,Meta&rsquo;s new Ray-Ban smart glasses have tw...,https://www.theverge.com/news/780012/meta-ray-...,Meta has announced a new generation of its Ra...,2025-09-18 00:07:37,402
6158,Beyond “Looks Good to Me”: How to Quantify LLM...,https://pub.towardsai.net/beyond-looks-good-to...,The Production HurdleThe greatest challenge fa...,2025-09-17 21:31:29,1315
7465,AI-Driven “Vibe Coding”: A New Era of Developm...,https://pub.towardsai.net/ai-driven-vibe-codin...,Is AI developer a “Vibe Coder”?The term “vibe ...,2025-09-15 15:16:21,1776


In [61]:
# Drop duplicated posts
innoreader_all = innoreader_all.drop_duplicates(subset=['url'])
innoreader_all['date_published'] = pd.to_datetime(innoreader_all['date_published'], format='mixed', utc=True)
innoreader_all = innoreader_all.sort_values(by=['date_published'], ascending=False)

print("Rows total: ", len(innoreader_all))
print('From: ', min(innoreader_all['date_published']))
print('To: ', max(innoreader_all['date_published']))
print("Tokens total: ", innoreader_all['token_count'].sum())

innoreader_all

Rows total:  2803
From:  2025-08-02 02:01:31+00:00
To:  2025-10-28 15:00:33+00:00
Tokens total:  3654020


,title,url,content_html,date_published,token_count
4,Finding return on AI investments across indust...,https://www.technologyreview.com/2025/10/28/11...,The market is officially three years post Chat...,2025-10-28 15:00:33+00:00,1241
23,OpenAI completed its for-profit restructuring ...,https://www.theverge.com/news/807875/openai-mi...,Microsoft Chairman and Chief Executive Office...,2025-10-28 13:38:55+00:00,557
32,Tech chiefs tell Trump to call off troops – wi...,https://www.theguardian.com/technology/2025/oc...,In this week’s newsletter: Silicon Valley bill...,2025-10-28 13:22:19+00:00,253
35,Adobe Max 2025: all the latest creative tools ...,https://www.theverge.com/news/807867/adobe-max...,Adobe has kicked off its annual Max design co...,2025-10-28 13:16:17+00:00,245
43,Fast vs. Slow: How (and When) to Make Models T...,https://pub.towardsai.net/fast-vs-slow-how-and...,Everyone in the Room Went “Hmm”! Except me…Our...,2025-10-28 12:26:53+00:00,1921
...,...,...,...,...,...
920,"NYT Connections hints today: Clues, answers fo...",https://mashable.com/article/nyt-connections-h...,Connections is the one of the most popular New...,2025-08-02 03:49:32+00:00,758
917,"Wordle today: Answer, hints for August 2, 2025",https://mashable.com/article/wordle-today-answ...,"Oh hey there! If you're here, it must be time ...",2025-08-02 03:49:32+00:00,747
919,"NYT Strands hints, answers for August 2, 2025",https://mashable.com/article/strands-nyt-hint-...,"If you're reading this, you're looking for a l...",2025-08-02 03:49:32+00:00,436
922,The Whole Story of MDP in RL,https://pub.towardsai.net/the-whole-story-of-m...,Understand what Markov Decision Processes are ...,2025-08-02 02:01:31+00:00,1697


## Save combined dataset

In [62]:
#Save data

file_path = 'C:\\Users\\Denis_Davydov2\\OneDrive - EPAM\\Prophet_AI_docs\\Datasets\\AI_skills\\'
file_name = f'Inoreader_all_{date.today()}.csv'

innoreader_all.to_csv(file_path + file_name, index=False)
print(f"File {file_name} uploaded: ", date.today())

print('File uploaded successfully')
print(innoreader_all.shape)

File Inoreader_all_2025-10-29.csv uploaded:  2025-10-29
File uploaded successfully
(2803, 5)
